In [26]:
import os
import cv2
import numpy as np
import caml_core as core

In [27]:
scanner_T_table = core.io.load_pose("data/motioncam_scans/scanner_T_tabletop.yaml")

In [28]:
background = core.io.load_snapshot("data/motioncam_scans/background").get_observations()[0]
observation = core.io.load_snapshot("data/motioncam_scans/snapshot_4").get_observations()[0]

observation.point_cloud.show()

In [29]:
mask = np.logical_or(
    (background.depth_image.image - observation.depth_image.image) > 0.005,
    np.logical_and(observation.depth_image.image > 0, background.depth_image.image == 0)
)

mask = core.geometry.Image(mask.astype(np.float64))
kernel = np.ones((5, 5))

mask = mask.erode(kernel=kernel, iterations=3)
mask = mask.dilate(kernel=kernel, iterations=3)

mask = mask.dilate(kernel=kernel, iterations=3)
mask = mask.erode(kernel=kernel, iterations=5)

mask.show()

In [30]:
masked_point_cloud = observation.point_cloud.select_by_image_mask(
    mask=mask,
    extrinsics=core.geometry.Pose(),
    intrinsics=observation.get_intrinsics(),
    distortions=observation.get_distortion_coefficients(),
)
masked_point_cloud.show()

In [31]:

cleaned_pc, _ = masked_point_cloud.as_open3d().remove_radius_outlier(50, 0.005)
cleaned_pc, _ = cleaned_pc.remove_statistical_outlier(50, 2.0)

obb = cleaned_pc.get_minimal_oriented_bounding_box()

sorted_indicies = np.argsort(obb.extent)

desired_x = -scanner_T_table.get_rotation_matrix()[:, 2]
desired_z = obb.R[:, sorted_indicies[0]]
desired_z = core.math.normalize_vector(desired_z - np.dot(desired_z, desired_x) * desired_x)
desired_R = core.math.build_rotation_matrix(x_axis=desired_x, z_axis=desired_z)

center = core.geometry.Pose(translation=obb.center, rotation_matrix=desired_R)

cleaned_point_cloud = core.geometry.PointCloud(cleaned_pc)

core.visualization.draw_geometries_with_hiding([], [center, cleaned_point_cloud])

In [32]:
# body_support = core.io.load_part_template(os.path.join(core.io.get_oasis_path(), "CAMLPartTemplates/demo/demo_v2/body_support"))
# body_support.show()